## SCROLL THROUGH BEFORE RUNNING ANY CELLS

In [1]:
import pandas as pd
import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter

In [ ]:
# SET IF WANTING TO DISPLAY FULL DATAFRAME (after simplifying step)

#pd.set_option('display.max_rows', 500)

In [2]:
# read in the dataset 
df = pd.read_json('animal_news_database.json', lines = True)

In [17]:
df

,_id,common_name,scientific_name,distribution,iucn_status,articles,date_saved,article_amt
0,{'$oid': '5eb530eeccc46a6a739577a2'},Campbell's alligator lizard,Abronia campbelli,[Guatemala],Critically Endangered,{'0': {'text': 'A 3-foot-long (1 meter) green ...,2020-06-18,2.0
1,{'$oid': '5eb53116ccc46a6a739577a6'},Southern plains gray langur,Semnopithecus dussumieri,[India],Least Concern,"{'3': {'text': 'On Thursday, a Nilgai was foun...",2020-06-18,1.0
2,{'$oid': '5eb53156ccc46a6a739577ab'},Natal giant cycad,Encephalartos natalensis,[South Africa],Near Threatened,{'0': {'text': 'A GUIDED walk entitled Interes...,2020-06-18,1.0
3,{'$oid': '5eb53162ccc46a6a739577ad'},Bengal hanuman langur,Semnopithecus entellus,"[Bangladesh, India, Nepal, Pakistan]",Least Concern,{'1': {'text': 'Advertisement From the tropic...,2020-06-18,1.0
4,{'$oid': '5eb53171ccc46a6a739577af'},Mongolian Wild Ass,Equus hemionus hemionus,"[China, Kazakhstan, Mongolia, Russian Federation]",NA,{'0': {'text': 'Exploding demand for cashmere ...,2020-06-18,5.0
...,...,...,...,...,...,...,...,...
2265,{'$oid': '5ed346f48089be8ca72beb7c'},Chiru,Pantholops hodgsonii,"[China, India, Nepal]",Near Threatened,{},2020-06-18,0.0
2266,{'$oid': '5ed347c88089be8ca72beb80'},Himalayan Monal,Lophophorus impejanus,"[Afghanistan, Bhutan, China, India, Myanmar, N...",Least Concern,{},2020-06-18,0.0
2267,{'$oid': '5ed348de8089be8ca72beb85'},Chinese Monal,Lophophorus lhuysii,[China],VU,{'5': {'text': 'A giant panda was captured by ...,2020-06-18,1.0
2268,{'$oid': '5ed349928089be8ca72beb86'},Chimpanzee,Pan troglodytes,"[Angola, Benin, Burkina Faso, Burundi, Cameroo...",EN,{'0': {'text': 'The Chimpanzee Sanctuary & Wil...,2020-06-18,5.0


In [11]:
# Drop Twitter handles as they are not species names
df = df[df.scientific_name.str.contains('@') == False]
df.reset_index(drop = True, inplace = True)

In [4]:
# Turn distribution column into lists so that each country is its own item

for i, row in df.iterrows():
    
    if ',' in row['distribution']:
        d = row['distribution'].split(', ')
        
        df.at[i, 'distribution'] = d
        
    else:
        single = [row['distribution']]
        
        df.at[i, 'distribution'] = single


In [16]:
# Add new column for storing the values article amount found under each species type 

for i, row in df.iterrows():
    df.at[i, 'article_amt'] = len(row.articles.values())


C:\Hyapp\Miniconda3\envs\python-gis\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Hyapp\Miniconda3\envs\python-gis\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
# Ensure that the country counts are taken per species not per row (AKA per scientific_name!)
df.loc[df['scientific_name'].isin(df['scientific_name'].unique())]

,_id,common_name,scientific_name,distribution,iucn_status,articles,date_saved,article_amt
0,{'$oid': '5eb530eeccc46a6a739577a2'},Campbell's alligator lizard,Abronia campbelli,[Guatemala],Critically Endangered,{'0': {'text': 'A 3-foot-long (1 meter) green ...,2020-06-18,2.0
1,{'$oid': '5eb53116ccc46a6a739577a6'},Southern plains gray langur,Semnopithecus dussumieri,[India],Least Concern,"{'3': {'text': 'On Thursday, a Nilgai was foun...",2020-06-18,1.0
2,{'$oid': '5eb53156ccc46a6a739577ab'},Natal giant cycad,Encephalartos natalensis,[South Africa],Near Threatened,{'0': {'text': 'A GUIDED walk entitled Interes...,2020-06-18,1.0
3,{'$oid': '5eb53162ccc46a6a739577ad'},Bengal hanuman langur,Semnopithecus entellus,"[Bangladesh, India, Nepal, Pakistan]",Least Concern,{'1': {'text': 'Advertisement From the tropic...,2020-06-18,1.0
4,{'$oid': '5eb53171ccc46a6a739577af'},Mongolian Wild Ass,Equus hemionus hemionus,"[China, Kazakhstan, Mongolia, Russian Federation]",NA,{'0': {'text': 'Exploding demand for cashmere ...,2020-06-18,5.0
...,...,...,...,...,...,...,...,...
2265,{'$oid': '5ed346f48089be8ca72beb7c'},Chiru,Pantholops hodgsonii,"[China, India, Nepal]",Near Threatened,{},2020-06-18,0.0
2266,{'$oid': '5ed347c88089be8ca72beb80'},Himalayan Monal,Lophophorus impejanus,"[Afghanistan, Bhutan, China, India, Myanmar, N...",Least Concern,{},2020-06-18,0.0
2267,{'$oid': '5ed348de8089be8ca72beb85'},Chinese Monal,Lophophorus lhuysii,[China],VU,{'5': {'text': 'A giant panda was captured by ...,2020-06-18,1.0
2268,{'$oid': '5ed349928089be8ca72beb86'},Chimpanzee,Pan troglodytes,"[Angola, Benin, Burkina Faso, Burundi, Cameroo...",EN,{'0': {'text': 'The Chimpanzee Sanctuary & Wil...,2020-06-18,5.0


In [76]:
# investigating where Province of China is coming from and what is means 
# Good syntax to keep for now
#df.loc[df.apply(lambda r: "Congo" in r["distribution"], axis=1)]

,scientific_name,distribution,iucn_status,article_amt
22,Loxodonta africana,"[Angola, Benin, Botswana, Burkina Faso, Burund...",Vulnerable,33.0
30,Loxodonta africana,"[Angola, Benin, Botswana, Burkina Faso, Burund...",Vulnerable,17.0
35,Loxodonta africana,"[Angola, Benin, Botswana, Burkina Faso, Burund...",Vulnerable,1.0
86,Eretmochelys imbricata,"[Albania, American Samoa, Anguilla (United Kin...",CR,34.0
112,Sousa teuszii,"[Angola, Benin, Cameroon, Congo, CÃ´te d'Ivoir...",CR,7.0
113,Sousa teuszii,"[Angola, Benin, Cameroon, Congo, CÃ´te d'Ivoir...",CR,3.0
161,Mandrillus sphinx,"[Cameroon, Congo, Equatorial Guinea, Gabon]",VU,14.0
166,Physeter macrocephalus,"[American Samoa, Angola, Anguilla (United King...",VU,2.0
167,Physeter macrocephalus,"[American Samoa, Angola, Anguilla (United King...",VU,1.0
183,Physeter macrocephalus,"[American Samoa, Angola, Anguilla (United King...",VU,13.0


In [44]:
# Keep relevant columns 
df = df.drop(['_id', 'common_name', 'articles', 'date_saved'], axis=1)
# Aggregate the article totals for each species and distributions listed for a species (keep first instance as each repition is the same)
new_df = df.groupby(['scientific_name', 'iucn_status'], as_index=False, sort=False).agg({'article_amt':'sum', 'distribution':'first'})
# Display simplified df
new_df

,scientific_name,iucn_status,article_amt,distribution
0,Abronia campbelli,Critically Endangered,2.0,[Guatemala]
1,Semnopithecus dussumieri,Least Concern,1.0,[India]
2,Encephalartos natalensis,Near Threatened,1.0,[South Africa]
3,Semnopithecus entellus,Least Concern,109.0,"[Bangladesh, India, Nepal, Pakistan]"
4,Equus hemionus hemionus,NA,5.0,"[China, Kazakhstan, Mongolia, Russian Federation]"
...,...,...,...,...
580,Sauromalus varius,Vulnerable,0.0,[Mexico]
581,Brachyteles hypoxanthus,CR,0.0,[Brazil]
582,Lontra provocax,EN,0.0,"[Argentina, Chile]"
583,Strombocactus disciformis,VU,0.0,[Mexico]


In [48]:
# Using counter we can determine how many times each country is listed under a species' native distribution 
# AKA which countries/regions have higher amounts of Appendix 1 species 

c = Counter(new_df['distribution'].explode())

In [49]:
# create a list based on the country entries of each row found in the distribution column
country_list = list(c.items())

In [50]:
# create a dataframe focusing on the country names and # of times they are listed under a species' range dist. 
countries = pd.DataFrame(country_list, columns = ['name', 'freq'])

In [58]:
countries

,name,freq
0,Guatemala,32
1,India,124
2,South Africa,47
3,Bangladesh,74
4,Nepal,61
5,Pakistan,64
6,China,108
7,Kazakhstan,27
8,Mongolia,20
9,Russian Federation,46


In [65]:
# In order to run the countries through a geolocater, individual errors in names need to be fixed manually 

countries['name'].iloc[37] = "Côte d'Ivoire"
countries['name'].iloc[206] = "Réunion"
countries['name'].iloc[175] = "Curaçao"
countries['name'].iloc[154] = "French Guiana"
countries['name'].iloc[241] = "Sudan"
countries['name'].iloc[229] = "Svalbard and Jan Mayen"
# Latin name Georgia kept returning the state of Georgia in USA in geolocator. This was solved by inputting the name in Georgian language
countries['name'].iloc[94] = "საქართველო"
# Geolocator was unable to fetch country names (Iran) with additional titles, so they were removed
countries['name'] = countries['name'].str.replace("\(Islamic Republic of\)", "")
# Removing the substring of (United Kingdom) from all islands 
countries['name'] = countries['name'].str.replace("\(United Kingdom\)", "")


# drop any countries that remain unnamed 
countries.drop(countries.index[countries['name'] == 'NA'], inplace = True)

In [60]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim


In [66]:
# Import the geocoding tool
from geopandas.tools import geocode

# Geocode countries using Nominatim. Remember to provide a custom "application name" in the user_agent parameter!
geo = geocode(countries['name'], provider='nominatim', user_agent='redalisa', timeout=4)
geo.head()

,geometry,address
0,POINT (-90.12265 15.68710),Guatemala
1,POINT (78.66774 22.35111),India
2,POINT (24.99164 -28.81662),South Africa
3,POINT (90.29344 24.47693),বাংলাদেশ
4,POINT (84.09171 28.10839),नेपाल


In [67]:
# Display df
geo

,geometry,address
0,POINT (-90.12265 15.68710),Guatemala
1,POINT (78.66774 22.35111),India
2,POINT (24.99164 -28.81662),South Africa
3,POINT (90.29344 24.47693),বাংলাদেশ
4,POINT (84.09171 28.10839),नेपाल
5,POINT (71.24750 30.33084),پاکستان
6,POINT (104.99993 35.00007),中国
7,POINT (65.20932 47.22861),Қазақстан
8,POINT (103.84997 46.82504),Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ
9,POINT (97.74531 64.68631),Россия


In [68]:
# join the two dataframes (in order to bring the # of mentions each country has under species' dist)
maat = geo.join(countries)
maat

,geometry,address,name,freq
0,POINT (-90.12265 15.68710),Guatemala,Guatemala,32
1,POINT (78.66774 22.35111),India,India,124
2,POINT (24.99164 -28.81662),South Africa,South Africa,47
3,POINT (90.29344 24.47693),বাংলাদেশ,Bangladesh,74
4,POINT (84.09171 28.10839),नेपाल,Nepal,61
5,POINT (71.24750 30.33084),پاکستان,Pakistan,64
6,POINT (104.99993 35.00007),中国,China,108
7,POINT (65.20932 47.22861),Қазақстан,Kazakhstan,27
8,POINT (103.84997 46.82504),Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ,Mongolia,20
9,POINT (97.74531 64.68631),Россия,Russian Federation,46


In [81]:
# remove any empty geometry columns
#maat = maat[~maat.geometry.is_empty]

# Create lat and lon columns out of the geometry column
maat['lon'] = maat['geometry'].x
maat['lat'] = maat['geometry'].y

# remove unneccessary row, "Province of China" this came from earlier splitting of values in the distribution column by ','
# Original value was Taiwan, Province of China, so we can just keep the row that is listed as Taiwan, as the values are the same
# index location was 16
#maat = maat.drop([0, 16])


# Ensure that all countries in the dataframe are unique by runnning one more groupby function and aggregating the frequency(species) column
maat = maat.groupby(['geometry','address', 'lat', 'lon'], as_index=False, sort=False).agg({'freq':'sum', 'name':'last'})

# reset index (may not be necessary after groupby)
maat = maat.reset_index(drop=True)

In [82]:
maat

,geometry,address,lat,lon,freq,name
0,POINT (78.6677428 22.3511148),India,22.351115,78.667743,124,India
1,POINT (24.991639 -28.8166236),South Africa,-28.816624,24.991639,47,South Africa
2,POINT (90.2934413 24.4769288),বাংলাদেশ,24.476929,90.293441,74,Bangladesh
3,POINT (84.0917139 28.1083929),नेपाल,28.108393,84.091714,61,Nepal
4,POINT (71.247499 30.3308401),پاکستان,30.330840,71.247499,64,Pakistan
5,POINT (104.999927 35.000074),中国,35.000074,104.999927,108,China
6,POINT (65.20931969999999 47.2286086),Қазақстан,47.228609,65.209320,27,Kazakhstan
7,POINT (103.8499736 46.8250388),Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ,46.825039,103.849974,20,Mongolia
8,POINT (97.74530609999999 64.68631360000001),Россия,64.686314,97.745306,46,Russian Federation
9,POINT (95.99996520000001 17.1750495),မြန်မာ,17.175050,95.999965,88,Myanmar


In [84]:
# Save dataframe for easier continuation
maat.to_csv('maat_geofile.csv')

AttributeError: 'DataFrame' object has no attribute 'to_file'

## IF CONTINUING AT LATER POINT START FROM HERE

In [101]:
# Save dataframe for easier continuation
#maat.to_csv('countries_geo.csv')
# Read in saved dataframe as gdf
maat = gpd.read_file('countries_geo.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO", encoding='utf-8')
#maat.drop('Unnamed: 0', inplace = True, axis = 1)

In [14]:
# Read-in data 
from shapely import wkt

maat = pd.read_csv("countries_geo-updated.csv")

In [85]:
# ensure that geometry column can be converted into geopandas
#maat['geometry'] = maat['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(maat, crs='epsg:4326')

In [86]:
gdf

,geometry,address,lat,lon,freq,name
0,POINT (78.66774 22.35111),India,22.351115,78.667743,124,India
1,POINT (24.99164 -28.81662),South Africa,-28.816624,24.991639,47,South Africa
2,POINT (90.29344 24.47693),বাংলাদেশ,24.476929,90.293441,74,Bangladesh
3,POINT (84.09171 28.10839),नेपाल,28.108393,84.091714,61,Nepal
4,POINT (71.24750 30.33084),پاکستان,30.330840,71.247499,64,Pakistan
5,POINT (104.99993 35.00007),中国,35.000074,104.999927,108,China
6,POINT (65.20932 47.22861),Қазақстан,47.228609,65.209320,27,Kazakhstan
7,POINT (103.84997 46.82504),Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ,46.825039,103.849974,20,Mongolia
8,POINT (97.74531 64.68631),Россия,64.686314,97.745306,46,Russian Federation
9,POINT (95.99997 17.17505),မြန်မာ,17.175050,95.999965,88,Myanmar


In [87]:
# SAVE SHAPEFILE

gdf.to_file('maat-geo.shp')

In [88]:
# redefine the count column as integer 

gdf['freq'] = gdf['freq'].astype(int)

# rename column as count for better meaning
gdf = gdf.rename(columns={"freq": "species"})

In [90]:
gdf.head()

,geometry,address,lat,lon,species,name
0,POINT (78.66774 22.35111),India,22.351115,78.667743,124,India
1,POINT (24.99164 -28.81662),South Africa,-28.816624,24.991639,47,South Africa
2,POINT (90.29344 24.47693),বাংলাদেশ,24.476929,90.293441,74,Bangladesh
3,POINT (84.09171 28.10839),नेपाल,28.108393,84.091714,61,Nepal
4,POINT (71.24750 30.33084),پاکستان,30.330840,71.247499,64,Pakistan


In [91]:
# Create a world map to show distributions of users 
# NOT A GOOD VISUALIZATION need to demonstrate size according to species count value
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
#empty map
world_map= folium.Map(tiles="cartodbpositron")
m = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(maat)):
        lat = gdf.iloc[i]['lat']
        long = gdf.iloc[i]['lon']
        radius=5
        popup_text = """Country : {}<br>
                     Species Count : {}<br>"""
        popup_text = popup_text.format(gdf.iloc[i]['name'],
                                   gdf.iloc[i]['species']
                                   )
        folium.CircleMarker(location = [lat, long], radius = radius, popup= popup_text, fill =True).add_to(m)
#show the map
world_map

In [114]:
world_map.save("marker-cluster-map.html")

In [115]:
new_map = folium.Map(tiles = 'stamentoner')
heat_df = maat.loc[:,["lat","lon","freq"]]

heat_data = heat_df.values.tolist()

HeatMap(heat_data,radius=10).add_to(new_map)

new_map

In [116]:
new_map.save("heat-map.html")

## THIS MAP
### Displays according to value 

In [100]:
b = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

# add marker one by one on the map

# Code to display countries only with certain value higher than a specified value
#for i in range(0,len(maat)):
#if maat.iloc[i]['freq'] > 100:
#print(maat.iloc[i]['freq'])

for i in range(len(maat)):
    lat = gdf.iloc[i]['lat']
    long = gdf.iloc[i]['lon']
    radius=float(gdf.iloc[i]['species'])*4000
    popup_text = """Country : {}<br>
                     Species Count : {}<br>"""
    popup_text = popup_text.format(gdf.iloc[i]['name'],
                                   gdf.iloc[i]['species']
                                   )
    folium.Circle(location = [lat, long], radius = radius, popup= popup_text, color= 'red',fill =True).add_to(b)
        
#

# Show the map again
b

In [101]:
b.save("species-distribution-count.html")